<a href="https://colab.research.google.com/github/michaelwbrigham/postdoc_docking/blob/main/AutoDocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Docking Trials
Started 12 Nov 25

# Set Up

In [ ]:
%pip install -q condacolab
import condacolab
condacolab.install()

Traceback (most recent call last):
  File "/usr/local/bin/pip", line 7, in <module>
    from pip import main
ImportError: No module named pip
✨🍰✨ Everything looks OK!


In [ ]:
%pip install -q pdb2pqr meeko py3Dmol kora rdkit gimmi
!conda install -q -c bioconda mgltools openbabel

!wget --no-check-certificate https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.7/vina_1.2.7_linux_x86_64
!chmod +x vina_1.2.7_linux_x86_64
!mv vina_1.2.7_linux_x86_64 vina
%alias vina /content/vina

!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static
!chmod u+x smina.static
!mv smina.static smina
%alias smina /content/smina

!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
!chmod +x gnina
%alias gnina /content/gnina

import os
from datetime import datetime

Traceback (most recent call last):
  File "/usr/local/bin/pip", line 7, in <module>
    from pip import main
ImportError: No module named pip
Traceback (most recent call last):
  File "/usr/local/bin/conda", line 12, in <module>
    from conda.cli import main
ImportError: No module named conda.cli
--2025-12-02 13:10:22--  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.7/vina_1.2.7_linux_x86_64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/258054635/2c1a7c52-9d1a-4b51-acfa-60c2e805c70e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T13%3A48%3A14Z&rscd=attachment%3B+filename%3Dvina_1.2.7_linux_x86_64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T12%3A47%3A26Z&ske=2025-12-

In [ ]:
def prepare_receptor(out_path, rec_path):
  rec_file = os.path.splitext(os.path.basename(rec_path))[0]

  %cd {out_path}
  !pdb2pqr --ff=AMBER --keep-chain --with-ph=7.4 {rec_path} {rec_file}.pqr
  !prepare_receptor4.py -r {rec_file}.pqr -o {rec_file}.pdbqt -C -U nphs_lps -v

  print(f"Created {rec_file}.pdbqt - this will be used for docking.")

  return f"{rec_file}.pdbqt"

def prepare_ligand_babel_closestconformer(out_path, new_lig_smiles, lig_name, ori_lig_path):
  %cd {out_path}

  with open(f"{lig_name}.smiles","w") as f:
    f.write(new_lig_smiles)

  print(f"Created {lig_name}.smiles")

  !obabel -:"{new_lig_smiles}" -O {lig_name}_temp.mol2 --gen3d --best --canonical -p 7.4
  print(f"Created {lig_name}_temp.mol2 - 3D coordinates from smiles, pH 7.4, not minimised.")

  !obabel {lig_name}_temp.mol2 -O {lig_name}_confs.sdf --conformer --nconf 100 --score rmsd --writeconformers
  print(f"Created {lig_name}_confs.sdf - 100 conformers based on {lig_name}_temp.mol2")

  !obabel {ori_lig_path} {lig_name}_confs.sdf -O {lig_name}.mol2 --align --smallest 2 rmsd
  !obabel {lig_name}.mol2 -omol2 -O {lig_name}_.mol2 -m
  print(f"Created {lig_name}_1.mol2 and {lig_name}_2.mol2 - _1 is the original ligand file and _2 is the computational ligand closed to the ligand file.")

  !prepare_ligand4.py -l {lig_name}_2.mol2 -o {lig_name}.pdbqt -U nphs_lps -v
  print(f"Created {lig_name}.pdbqt - this will be used for docking.")

def prepare_ligand(out_path, new_lig_smiles, lig_name, ori_lig_path):
  %cd {out_path}

  with open(f"{lig_name}.smiles","w") as f:
    f.write(new_lig_smiles)

  print(f"Created {lig_name}.smiles")

  !obabel -:"{new_lig_smiles}" -O {lig_name}_temp.mol2 --gen3d --best --canonical -p 7.4
  !prepare_ligand4.py -l {lig_name}_2.mol2 -o {lig_name}.pdbqt -U nphs_lps -v

  return f"{lig_name}.pdbqt"

def view_docking(rec_path,ori_lig_path,new_lig_path,box_center,box_dimensions):
  mview = py3Dmol.view(900, 900)
  rec_mol = open(rec_path, 'r').read()
  old_lig_mol = open(ori_lig_path, 'r').read()
  best_new_lig_path = open(best_new_lig_path, 'r').read()

  mview.addModel(rec_mol,'pdb')
  mview.setStyle({'model':1},{'cartoon': {'color':'grey'}})

  mview.addModel(old_lig_mol,'pdb')
  mview.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})

  mview.addModel(best_new_lig_path,'pdb')
  mview.setStyle({'model':2},{'stick':{'colorscheme':'blueCarbon'}})

  mview.addBox({'center':{'x':box_center[0],'y':box_center[1],'z':box_center[2]},
                'dimensions': {'w':box_dimensions[0],'h':box_dimensions[1],'d':box_dimensions[2]},'color':'yellow','opacity': 0.5})

  mview.setBackgroundColor('0xffffff')
  mview.zoomTo()
  mview.show()

def create_config_file(receptor_file, ligand_file, box_center, box_dimensions):
  %cd {out_path}

  with open(f"config_singledock","w") as f:
    f.write("#CONFIGURATION FILE (options not used are commented) \n")
    f.write("\n")
    f.write("#INPUT OPTIONS \n")
    f.write(f"receptor = {receptor_file} \n")
    f.write(f"ligand = {ligand_file} \n")
    f.write(f"#SEARCH SPACE CONFIGURATIONS \n")
    f.write(f"#Center of the box (values bxi, byi and bzi) \n")
    f.write(f"center_x = {box_center[0]} \n")
    f.write(f"center_y = {box_center[1]} \n")
    f.write(f"center_z = {box_center[2]} \n")
    f.write(f"#Size of the box (values bxf, byf and bzf) \n")
    f.write(f"size_x = {box_dimensions[0]} \n")
    f.write(f"size_y = {box_dimensions[1]} \n")
    f.write(f"size_z = {box_dimensions[2]} \n")
    f.write("#OUTPUT OPTIONS \n")
    f.write("#out = \n")
    f.write("#log = \n")
    f.write("\n")
    f.write("#OTHER OPTIONS \n")
    f.write("cpu =  8\n")
    f.write("exhaustiveness = 16\n")
    f.write("#num_modes = \n")
    f.write("#energy_range = \n")
    f.write("#seed = ")

  print("Created config_singledock file - this will configure the docking run.")

  return "config_singledock"

def run_docking(out_path, config_file, vina_run_bool, smina_run_bool, gnina_run_bool):
  %cd {out_path}

  if gnina_run_bool:
    %gnina --config {config_file} --out lig_out_gnina.pdbqt

  if vina_run_bool:
    %vina --config {config_file} --out lig_out_vina.pdbqt

  if smina_run_bool:
    %smina --config {config_file} --out lig_out_smina.pdbqt

# Docking Setup

In [ ]:
# @markdown ## Output path
gen_out_path = "/content/Output" # @param {"type":"string","placeholder":"/content/251112_Docking/Output"}
out_path = os.path.join(gen_out_path,datetime.now().strftime("%y%m%d_%H%M%S"))
os.makedirs(out_path,exist_ok=True)

# @markdown ---
# @markdown ## Receptor
rec_path = "/content/Input/g11_loops_modelled.pdb" # @param {"type":"string","placeholder":"/content/251112_Docking/Input"}
# @markdown ---
# @markdown ## Ligand
lig_name = "FR" # @param {"type":"string","placeholder":"g11"}
ori_lig_path = "/content/Input/251114_FR.pdb" # @param {"type":"string","placeholder":"/content/Input/251114_FR.pdb"}
new_lig_smiles = "N1(C(=C)C(=O)N[C@H](C(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)O[C@H](C(C)C)[C@H](NC(=O)C)C(=O)O[C@@H](C1=O)Cc1ccccc1)[C@@H](C)OC)C)[C@@H](C(C)C)OC(=O)[C@@H](NC(=O)CC)[C@@H](C(C)C)O)C)C)C)C" # @param {"type":"string","placeholder":"N1(C(=C)C(=O)N[C@H](C(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)O[C@H](C(C)C)[C@H](NC(=O)C)C(=O)O[C@@H](C1=O)Cc1ccccc1)[C@@H](C)OC)C)[C@@H](C(C)C)OC(=O)[C@@H](NC(=O)CC)[C@@H](C(C)C)O)C)C)C)C"}
# @markdown ---
# @markdown ## Box Set Up
centre_x = -39.3 # @param {"type":"number","placeholder":"-39.4"}
centre_y = 2.9 # @param {"type":"number","placeholder":"2.9"}
centre_z = -8.3 # @param {"type":"number","placeholder":"-8.1"}
# @markdown #
boxsize_x = 17 # @param {"type":"number","placeholder":"26.4"}
boxsize_y = 22 # @param {"type":"number","placeholder":"20.7"}
boxsize_z = 17 # @param {"type":"number","placeholder":"17.4"}
# @markdown ---
vina_run_bool = True # @param {type:"boolean"}
smina_run_bool = True # @param {type:"boolean"}
gnina_run_bool = True # @param {type:"boolean"}


# Receptor preparation

In [ ]:
rec_file = prepare_receptor(out_path=out_path,
                            rec_path=rec_path)

# Ligand prep

In [ ]:
lig_file = prepare_ligand(out_path = out_path,
                          new_lig_smiles = new_lig_smiles,
                          lig_name = lig_name,
                          ori_lig_path = ori_lig_path)

In [ ]:
view_docking(rec_path = rec_path,
             ori_lig_path = ori_lig_path,
             new_lig_path = lig_file,
             box_center = (centre_x, centre_y, centre_z),
             box_dimensions = (boxsize_x, boxsize_y, boxsize_z))

In [ ]:
create_config_file(receptor_file = rec_file,
                   ligand_file = lig_file,
                   box_center = (centre_x, centre_y, centre_z),
                   box_dimensions = (boxsize_x, boxsize_y, boxsize_z))

In [ ]:
run_docking(out_path = out_path,
            config_file = con_file,
            vina_run_bool = vina_run_bool,
            smina_run_bool = smina_run_bool,
            gnina_run_bool = gnina_run_bool):

AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V